In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #Gives us Graphics
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten


In [2]:
# Load data
url = 'https://raw.githubusercontent.com/beespinosa1/Inter/main/Escenario3.csv'
df = pd.read_csv(url)


In [3]:
# Rename columns
names =['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur',
        'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service',
        'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb',
        'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
        'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
        'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
        'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
        'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
        'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'Label']
df = pd.read_csv(url, names=names, low_memory=False)


In [4]:
# Shuffle data
dataframe = df.reindex(np.random.permutation(df.index)).copy()


In [5]:
# Normalize data
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df


In [6]:
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys)-1]].copy()
x_normalized = dfNormalize(data_to_process)


In [7]:
# Split the dataset into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_normalised, y, test_size=0.3)


NameError: ignored

In [ ]:
# Convert labels to binary format
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y_train = encoder.transform(y_train)
dummy_y_train = np_utils.to_categorical(encoded_Y_train)

encoder.fit(y_test)
encoded_Y_test = encoder.transform(y_test)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)

In [ ]:
# Define the model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()

model.add(Conv1D(64, 3, padding='same', input_shape=(x_normalised.shape[1],1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(128, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(dummy_y_train.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train.values.reshape(X_train.shape[0],X_train.shape[1],1), dummy_y_train, epochs=10, batch_size=32, validation_data=(X_test.values.reshape(X_test.shape[0],X_test.shape[1],1), dummy_y_test))


In [ ]:
# Evaluate the model
score = model.evaluate(X_test.values.reshape(X_test.shape[0],X_test.shape[1],1), dummy_y_test, batch_size=32)
print('Test loss:', score[0])
print('Test accuracy:', score[1])